In [111]:
import FinanceDataReader as fdr
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import plotly.express as px

# 271060 : KODEX 3대농산물선물(H) - 옥수수, 콩, 밀
# 전쟁의 영향력을 알기 위해 '러시아의 우크라이나 침공이 언급되는 시점 및 전년동기대비 가격
beforewar = pd.read_csv('../csv/beforewar.csv')
afterwar = pd.read_csv('../csv/afterwar.csv')

# 연도 제외 데이터로 처리
# beforewar['Date'] = beforewar['Date'].str[5:]
# afterwar['Date'] = afterwar['Date'].str[5:]

beforewar['Date'] = pd.to_datetime(beforewar['Date'])
afterwar['Date'] = pd.to_datetime(afterwar['Date'])

beforewar['Date'] = beforewar['Date'].dt.strftime('%m/%d')
afterwar['Date'] = afterwar['Date'].dt.strftime('%m/%d')

# 컬럼명 변경
beforewar = beforewar.rename(columns={'Close':'Before_Close'})
afterwar = afterwar.rename(columns={'Close':'After_Close'})

# 병합
war_produce = pd.merge(beforewar, afterwar, how='inner', on='Date')
war_produce[['Date', 'Before_Close', 'After_Close']]

# 시각화
fig = px.line(war_produce, x='Date', y=["Before_Close","After_Close"])
fig.show()